### Dropped from na02approximation.ipynb

Move to M104C


![Chebychev polynomial 3](https://jhparkyb.github.io/resources/notes/na/104ASlides_Approximation021.png)

##### Divided differences

![Divided differences 1](https://jhparkyb.github.io/resources/notes/na/104ASlides_Approximation026.png)

![Divided differences 2](https://jhparkyb.github.io/resources/notes/na/104ASlides_Approximation027.png)

![Divided differences 3](https://jhparkyb.github.io/resources/notes/na/104ASlides_Approximation028.png)

##### Horner's algorithm: efficient evaluation of interpolation

Efficient way of evaluating polynomails.

![Example of Horner's algorithm](https://jhparkyb.github.io/resources/notes/na/104ASlides_Approximation010.png)

![Finding interpolating polynomial in the Newton form 2](https://jhparkyb.github.io/resources/notes/na/104ABoardWork_Approximation002.png)